In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys


# Logistic Regression classifier

https://pythonkim.tistory.com/20?category=573319

In [2]:
x_data = [[1, 2],
          [2, 3],
          [3, 1],
          [4, 3],
          [5, 3],
          [6, 2]]
y_data = [[0],
          [0],
          [0],
          [1],
          [1],
          [1]]


In [3]:
X = tf.placeholder(tf.float32, shape=[None,2])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([2,1]))
b = tf.Variable(tf.random_normal([1]))
# Hypothessis using sigmoid
hypot = tf.sigmoid(tf.matmul(X,W)+b)
# cost/loss function
cost = -tf.reduce_mean(Y*tf.log(hypot)+(1-Y)*tf.log(1-hypot))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(10000):
    sess.run(train, feed_dict = {X:x_data, Y:y_data})

In [6]:
pred =  tf.cast(hypot > 0.5, dtype=tf.float32)
sess.run(pred, feed_dict = {X:x_data, Y:y_data})
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, Y), dtype=tf.float32))

h, p, a = sess.run([hypot, pred, accuracy], feed_dict = {X:x_data, Y:y_data})
print("가설 : ", h, "\n예측 : ", p, "\n정확도 : ", a)

가설 :  [[0.0239348 ]
 [0.14845188]
 [0.2704267 ]
 [0.7975896 ]
 [0.9493262 ]
 [0.98348826]] 
예측 :  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
정확도 :  1.0


# Softmax

## 어떤~값이 들어가면 001, 010, 100 이 정답이다

In [7]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], 
                                                        [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

In [8]:
X = tf.placeholder(tf.float32, shape = [None, 4])
Y = tf.placeholder(tf.float32, shape = [None, 3])
# n, 행 4열  X  (a, b) 3 -> 4, 3
W = tf.Variable(tf.random_normal([4,3]))
b = tf.Variable(tf.random_normal([3]))

hypot = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypot), axis=1))

train  = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    sess.run(train, feed_dict = {X:x_data, Y:y_data})

In [10]:
result = sess.run(hypot, feed_dict = {X:[[1,11,7,9]]})
print(result)

[[1.9743282e-02 9.8024827e-01 8.4696503e-06]]


In [11]:
print(sess.run(tf.arg_max(result, 1)))

[1]


# 함수쓰기

In [13]:
X = tf.placeholder(tf.float32, shape = [None, 4])
Y = tf.placeholder(tf.float32, shape = [None, 3])
# n, 행 4열  X  (a, b) 3 -> 4, 3
W = tf.Variable(tf.random_normal([4,3]))
b = tf.Variable(tf.random_normal([3]))

# hypot = tf.nn.softmax(tf.matmul(X, W) + b)

# cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypot), axis=1))

logit = tf.matmul(X, W) + b
hyhot = tf.nn.softmax(logit)
cost = tf.nn.softmax_cross_entropy_with_logits(logits = logit, labels = y_data)

train  = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# 사례 1 : 동물 분류

- https://kjun.kr/475?category=754847 동물 분류하기


In [18]:
xy = np.loadtxt('data/data-04-zoo.csv',delimiter = ',', dtype = np.float32)
xy.shape

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [20]:
X = tf.placeholder(tf.float32, shape = [None, 16])
y = tf.placeholder(tf.int32, shape = [None, 1])
#원핫 이코딩
y_one_hot=tf.one_hot(y, 7)
#y_one_hot   << 하면 shape ( ?, 1, 7) 로 나옴
y_one_hot=tf.reshape(y_one_hot, [-1,7]) # 100, 1, 7 -> 100, 7

# W = tf.Variable(tf.random_normal([16,1]))
# b = tf.Variable(tf.random_normal([1]))


W = tf.Variable(tf.random_normal([16,7]))
b = tf.Variable(tf.random_normal([7]))

#알고리즘 부분
logit = tf.matmul(X, W) + b
hyhot = tf.nn.softmax(logit)
cost = tf.nn.softmax_cross_entropy_with_logits(logits = logit, labels = y_one_hot)

train  = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
# 실행부분 200번수행
sess = tf.Session()
sess.run(tf.global_variables_initializer())

pred = tf.argmax(hypot, 1)
correct = tf.equal(pred, tf.argmax(y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


for step in range(200):
    sess.run(train, feed_dict={X:x_data, y:y_data})
    
    if step % 10 == 0:
        loss, acc = sess.run([tf.reduce_mean(cost), accuracy], feed_dict={X:x_data, y:y_data}) # 비용 평균과 정확도
        print("step:{:5}\tLoss:{:.3f}\tAcc:{}".format(step, loss, acc))
#결과보기 

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float and shape [?,4]
	 [[node Placeholder_2 (defined at C:\Users\acorn\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'Placeholder_2':
  File "C:\Users\acorn\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\acorn\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\acorn\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\Users\acorn\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
    self.asyncio_loop.run_forever

In [21]:
# 데이터 가공 및 준비
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

# 텐서 그래프 준비
X = tf.placeholder(tf.float32, shape=[None, 16])
y = tf.placeholder(tf.int32, shape=[None, 1])



# 원래
# W = tf.Variable(tf.random_normal([16, 1]))
# b = tf.Variable(tf.random_normal([1]))

# one_hot 매서드 활용하면?
y_one_hot = tf.one_hot(y, 7)
y_one_hot = tf.reshape(y_one_hot, [-1,7])

# 가중치와 바이어너리
W = tf.Variable(tf.random_normal([16,7]))
b = tf.Variable(tf.random_normal([7]))



# 가설설정
logit = tf.matmul(X,W) + b
hypot = tf.nn.softmax(logit)

# 비용함수
cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit, labels = y_one_hot)

# 최소비용게산
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# 실행준비
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 성능 측정 준비
pred = tf.argmax(hypot, 1)                                # 예측
correct = tf.equal(pred, tf.argmax(y_one_hot, 1))         # 맞은 값 예측
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   # 정확도


# 훈련 실행

for step in range(200):
    sess.run(train, feed_dict={X:x_data, y:y_data})
    
    if step % 10 == 0:
        loss, acc = sess.run([tf.reduce_mean(cost), accuracy], feed_dict={X:x_data, y:y_data}) # 비용 평균과 정확도
        print("step:{:5}\tLoss:{:.3f}\tAcc:{}".format(step, loss, acc))

step:    0	Loss:3.974	Acc:0.3762376308441162
step:   10	Loss:0.508	Acc:0.8811880946159363
step:   20	Loss:0.316	Acc:0.9306930899620056
step:   30	Loss:0.234	Acc:0.9702970385551453
step:   40	Loss:0.191	Acc:0.9702970385551453
step:   50	Loss:0.163	Acc:0.9702970385551453
step:   60	Loss:0.143	Acc:0.9702970385551453
step:   70	Loss:0.127	Acc:0.9900990128517151
step:   80	Loss:0.115	Acc:0.9900990128517151
step:   90	Loss:0.105	Acc:0.9900990128517151
step:  100	Loss:0.096	Acc:0.9900990128517151
step:  110	Loss:0.089	Acc:1.0
step:  120	Loss:0.083	Acc:1.0
step:  130	Loss:0.077	Acc:1.0
step:  140	Loss:0.073	Acc:1.0
step:  150	Loss:0.068	Acc:1.0
step:  160	Loss:0.064	Acc:1.0
step:  170	Loss:0.061	Acc:1.0
step:  180	Loss:0.058	Acc:1.0
step:  190	Loss:0.055	Acc:1.0


### 사례 2 : MNIST 숫자 이미지 

 - https://goodtogreate.tistory.com/entry/MNIST-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%85%8B%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%86%90%EA%B8%80%EC%94%A8-%EC%9D%B8%EC%8B%9D-Deep-Neural-Network-%EA%B5%AC%ED%98%84
 구현코드

In [22]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("data/MNIST_data/", one_hot = True)



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official

In [26]:
type(mnist)
dir(mnist)
mnist.train.labels # 
mnist.train.num_examples #55000개 

55000

In [29]:
X = tf.placeholder(tf.float32, shape = [None, 784]) 
#열의크기 28x28, 28*28 두번째 인자로 해도됨
y = tf.placeholder(tf.float32, shape = [None, 10])
#모델 가중치 세팅 , set model weigts
W = tf.Variable(tf.random_normal([784,10]))
b = tf.Variable(tf.random_normal([10]))
# 

logit = tf.matmul(X,W) + b
hypot = tf.nn.softmax(logit)
# 비용함수
cost = tf.nn.softmax_cross_entropy_with_logits(logits = logit, labels = y)

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)


correct = tf.equal(tf.argmax(hypot, 1), tf.argmax(y, 1))         # 맞은 값 예측
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   # 정확도

sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 100

for epoch in range(15):
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, loss, acc = sess.run([train, tf.reduce_mean(cost), accuracy],
                                feed_dict = {X:batch_xs, y:batch_ys})
        print("epoch:{:5}\tLoss:{:.3f}\tAcc:{}".format(epoch, loss, acc))

epoch:    0	Loss:15.554	Acc:0.10000000149011612
epoch:    0	Loss:12.626	Acc:0.07999999821186066
epoch:    0	Loss:8.766	Acc:0.17000000178813934
epoch:    0	Loss:7.899	Acc:0.1899999976158142
epoch:    0	Loss:8.103	Acc:0.20999999344348907
epoch:    0	Loss:6.856	Acc:0.23000000417232513
epoch:    0	Loss:5.030	Acc:0.3400000035762787
epoch:    0	Loss:6.794	Acc:0.28999999165534973
epoch:    0	Loss:4.898	Acc:0.3400000035762787
epoch:    0	Loss:4.984	Acc:0.36000001430511475
epoch:    0	Loss:4.750	Acc:0.3499999940395355
epoch:    0	Loss:2.994	Acc:0.5199999809265137
epoch:    0	Loss:4.201	Acc:0.41999998688697815
epoch:    0	Loss:2.776	Acc:0.5400000214576721
epoch:    0	Loss:2.375	Acc:0.5600000023841858
epoch:    0	Loss:3.786	Acc:0.49000000953674316
epoch:    0	Loss:1.994	Acc:0.6299999952316284
epoch:    0	Loss:3.143	Acc:0.49000000953674316
epoch:    0	Loss:2.845	Acc:0.5299999713897705
epoch:    0	Loss:2.068	Acc:0.6100000143051147
epoch:    0	Loss:1.896	Acc:0.6200000047683716
epoch:    0	Loss:2.336

epoch:    0	Loss:0.764	Acc:0.8299999833106995
epoch:    0	Loss:0.812	Acc:0.8199999928474426
epoch:    0	Loss:0.580	Acc:0.8700000047683716
epoch:    0	Loss:1.008	Acc:0.7900000214576721
epoch:    0	Loss:0.559	Acc:0.8399999737739563
epoch:    0	Loss:0.723	Acc:0.8199999928474426
epoch:    0	Loss:1.345	Acc:0.7400000095367432
epoch:    0	Loss:0.802	Acc:0.8399999737739563
epoch:    0	Loss:0.936	Acc:0.8600000143051147
epoch:    0	Loss:0.573	Acc:0.8600000143051147
epoch:    0	Loss:0.876	Acc:0.800000011920929
epoch:    0	Loss:0.827	Acc:0.8600000143051147
epoch:    0	Loss:0.937	Acc:0.8399999737739563
epoch:    0	Loss:0.787	Acc:0.800000011920929
epoch:    0	Loss:0.949	Acc:0.7599999904632568
epoch:    0	Loss:0.796	Acc:0.8299999833106995
epoch:    0	Loss:0.957	Acc:0.8199999928474426
epoch:    0	Loss:0.899	Acc:0.7900000214576721
epoch:    0	Loss:1.107	Acc:0.800000011920929
epoch:    0	Loss:0.996	Acc:0.8199999928474426
epoch:    0	Loss:0.265	Acc:0.8999999761581421
epoch:    0	Loss:0.656	Acc:0.85000002

epoch:    0	Loss:0.436	Acc:0.8899999856948853
epoch:    0	Loss:0.577	Acc:0.8399999737739563
epoch:    0	Loss:0.700	Acc:0.8899999856948853
epoch:    0	Loss:0.720	Acc:0.8500000238418579
epoch:    0	Loss:0.540	Acc:0.8399999737739563
epoch:    0	Loss:0.957	Acc:0.7699999809265137
epoch:    0	Loss:0.786	Acc:0.8999999761581421
epoch:    0	Loss:0.834	Acc:0.7900000214576721
epoch:    0	Loss:0.532	Acc:0.8799999952316284
epoch:    0	Loss:0.767	Acc:0.7900000214576721
epoch:    0	Loss:0.630	Acc:0.8500000238418579
epoch:    0	Loss:0.651	Acc:0.8600000143051147
epoch:    0	Loss:0.678	Acc:0.8500000238418579
epoch:    0	Loss:0.642	Acc:0.8700000047683716
epoch:    0	Loss:0.707	Acc:0.8600000143051147
epoch:    0	Loss:0.790	Acc:0.8399999737739563
epoch:    0	Loss:0.756	Acc:0.8199999928474426
epoch:    0	Loss:0.885	Acc:0.8399999737739563
epoch:    0	Loss:0.727	Acc:0.8199999928474426
epoch:    0	Loss:0.627	Acc:0.8399999737739563
epoch:    0	Loss:0.242	Acc:0.9100000262260437
epoch:    0	Loss:0.924	Acc:0.80000

epoch:    0	Loss:0.776	Acc:0.8500000238418579
epoch:    0	Loss:0.473	Acc:0.9100000262260437
epoch:    0	Loss:0.308	Acc:0.9100000262260437
epoch:    0	Loss:0.539	Acc:0.8600000143051147
epoch:    0	Loss:0.354	Acc:0.9100000262260437
epoch:    1	Loss:0.750	Acc:0.8600000143051147
epoch:    1	Loss:0.928	Acc:0.8299999833106995
epoch:    1	Loss:0.612	Acc:0.8600000143051147
epoch:    1	Loss:0.561	Acc:0.8600000143051147
epoch:    1	Loss:0.379	Acc:0.9100000262260437
epoch:    1	Loss:0.942	Acc:0.8199999928474426
epoch:    1	Loss:0.400	Acc:0.8799999952316284
epoch:    1	Loss:0.736	Acc:0.8600000143051147
epoch:    1	Loss:0.390	Acc:0.8999999761581421
epoch:    1	Loss:0.505	Acc:0.8299999833106995
epoch:    1	Loss:0.383	Acc:0.9300000071525574
epoch:    1	Loss:0.456	Acc:0.9200000166893005
epoch:    1	Loss:0.901	Acc:0.8199999928474426
epoch:    1	Loss:0.959	Acc:0.8899999856948853
epoch:    1	Loss:0.624	Acc:0.8500000238418579
epoch:    1	Loss:0.565	Acc:0.8799999952316284
epoch:    1	Loss:0.495	Acc:0.87000

epoch:    1	Loss:0.763	Acc:0.8399999737739563
epoch:    1	Loss:0.500	Acc:0.8899999856948853
epoch:    1	Loss:0.419	Acc:0.8600000143051147
epoch:    1	Loss:0.639	Acc:0.8399999737739563
epoch:    1	Loss:0.573	Acc:0.8999999761581421
epoch:    1	Loss:0.560	Acc:0.8799999952316284
epoch:    1	Loss:1.051	Acc:0.7900000214576721
epoch:    1	Loss:0.653	Acc:0.8500000238418579
epoch:    1	Loss:0.417	Acc:0.8999999761581421
epoch:    1	Loss:0.444	Acc:0.8899999856948853
epoch:    1	Loss:0.319	Acc:0.8899999856948853
epoch:    1	Loss:0.877	Acc:0.8600000143051147
epoch:    1	Loss:0.574	Acc:0.8399999737739563
epoch:    1	Loss:0.573	Acc:0.8500000238418579
epoch:    1	Loss:0.469	Acc:0.8700000047683716
epoch:    1	Loss:0.425	Acc:0.8899999856948853
epoch:    1	Loss:0.752	Acc:0.8600000143051147
epoch:    1	Loss:0.431	Acc:0.8899999856948853
epoch:    1	Loss:0.569	Acc:0.8899999856948853
epoch:    1	Loss:0.725	Acc:0.8199999928474426
epoch:    1	Loss:0.393	Acc:0.8700000047683716
epoch:    1	Loss:0.352	Acc:0.93000

...

epoch:   13	Loss:0.213	Acc:0.9300000071525574
epoch:   13	Loss:0.403	Acc:0.9300000071525574
epoch:   13	Loss:0.251	Acc:0.9300000071525574
epoch:   13	Loss:0.226	Acc:0.9200000166893005
epoch:   13	Loss:0.363	Acc:0.9200000166893005
epoch:   13	Loss:0.501	Acc:0.8899999856948853
epoch:   13	Loss:0.282	Acc:0.9300000071525574
epoch:   13	Loss:0.316	Acc:0.8899999856948853
epoch:   13	Loss:0.350	Acc:0.8799999952316284
epoch:   13	Loss:0.348	Acc:0.9200000166893005
epoch:   13	Loss:0.132	Acc:0.949999988079071
epoch:   13	Loss:0.399	Acc:0.9100000262260437
epoch:   13	Loss:0.217	Acc:0.9300000071525574
epoch:   13	Loss:0.096	Acc:0.9800000190734863
epoch:   13	Loss:0.233	Acc:0.9399999976158142
epoch:   13	Loss:0.511	Acc:0.8700000047683716
epoch:   13	Loss:0.269	Acc:0.9399999976158142
epoch:   13	Loss:0.166	Acc:0.9700000286102295
epoch:   13	Loss:0.508	Acc:0.8700000047683716
epoch:   13	Loss:0.283	Acc:0.9399999976158142
epoch:   13	Loss:0.231	Acc:0.9200000166893005
epoch:   13	Loss:0.360	Acc:0.899999

epoch:   13	Loss:0.255	Acc:0.8999999761581421
epoch:   13	Loss:0.416	Acc:0.8700000047683716
epoch:   13	Loss:0.366	Acc:0.9200000166893005
epoch:   13	Loss:0.208	Acc:0.949999988079071
epoch:   13	Loss:0.371	Acc:0.8899999856948853
epoch:   13	Loss:0.186	Acc:0.9300000071525574
epoch:   13	Loss:0.292	Acc:0.9300000071525574
epoch:   13	Loss:0.238	Acc:0.9399999976158142
epoch:   13	Loss:0.207	Acc:0.9599999785423279
epoch:   13	Loss:0.556	Acc:0.8899999856948853
epoch:   13	Loss:0.291	Acc:0.9300000071525574
epoch:   13	Loss:0.390	Acc:0.949999988079071
epoch:   13	Loss:0.421	Acc:0.8700000047683716
epoch:   13	Loss:0.354	Acc:0.8899999856948853
epoch:   13	Loss:0.307	Acc:0.8899999856948853
epoch:   13	Loss:0.155	Acc:0.9599999785423279
epoch:   13	Loss:0.195	Acc:0.9300000071525574
epoch:   13	Loss:0.287	Acc:0.9300000071525574
epoch:   13	Loss:0.131	Acc:0.9700000286102295
epoch:   13	Loss:0.320	Acc:0.8899999856948853
epoch:   13	Loss:0.313	Acc:0.8899999856948853
epoch:   13	Loss:0.517	Acc:0.8799999

epoch:   14	Loss:0.292	Acc:0.9200000166893005
epoch:   14	Loss:0.256	Acc:0.8999999761581421
epoch:   14	Loss:0.312	Acc:0.8899999856948853
epoch:   14	Loss:0.221	Acc:0.9300000071525574
epoch:   14	Loss:0.091	Acc:0.9900000095367432
epoch:   14	Loss:0.232	Acc:0.9100000262260437
epoch:   14	Loss:0.392	Acc:0.9100000262260437
epoch:   14	Loss:0.565	Acc:0.8799999952316284
epoch:   14	Loss:0.245	Acc:0.8999999761581421
epoch:   14	Loss:0.411	Acc:0.9100000262260437
epoch:   14	Loss:0.265	Acc:0.9300000071525574
epoch:   14	Loss:0.264	Acc:0.9100000262260437
epoch:   14	Loss:0.309	Acc:0.8999999761581421
epoch:   14	Loss:0.118	Acc:0.9800000190734863
epoch:   14	Loss:0.185	Acc:0.9599999785423279
epoch:   14	Loss:0.190	Acc:0.949999988079071
epoch:   14	Loss:0.464	Acc:0.8999999761581421
epoch:   14	Loss:0.437	Acc:0.8799999952316284
epoch:   14	Loss:0.313	Acc:0.8899999856948853
epoch:   14	Loss:0.267	Acc:0.9700000286102295
epoch:   14	Loss:0.226	Acc:0.8999999761581421
epoch:   14	Loss:0.159	Acc:0.930000

epoch:   14	Loss:0.347	Acc:0.8999999761581421
epoch:   14	Loss:0.277	Acc:0.9200000166893005
epoch:   14	Loss:0.178	Acc:0.9300000071525574
epoch:   14	Loss:0.226	Acc:0.9399999976158142
epoch:   14	Loss:0.348	Acc:0.8999999761581421
epoch:   14	Loss:0.151	Acc:0.949999988079071
epoch:   14	Loss:0.310	Acc:0.8799999952316284
epoch:   14	Loss:0.293	Acc:0.8999999761581421
epoch:   14	Loss:0.474	Acc:0.8999999761581421
epoch:   14	Loss:0.423	Acc:0.9100000262260437
epoch:   14	Loss:0.172	Acc:0.9399999976158142
epoch:   14	Loss:0.344	Acc:0.8799999952316284
epoch:   14	Loss:0.130	Acc:0.9700000286102295
epoch:   14	Loss:0.152	Acc:0.9599999785423279
epoch:   14	Loss:0.458	Acc:0.8899999856948853
epoch:   14	Loss:0.303	Acc:0.8999999761581421
epoch:   14	Loss:0.308	Acc:0.9100000262260437
epoch:   14	Loss:0.203	Acc:0.9300000071525574
epoch:   14	Loss:0.163	Acc:0.9599999785423279
epoch:   14	Loss:0.323	Acc:0.9300000071525574
epoch:   14	Loss:0.475	Acc:0.9100000262260437
epoch:   14	Loss:0.329	Acc:0.910000

epoch:   14	Loss:0.224	Acc:0.9300000071525574
epoch:   14	Loss:0.165	Acc:0.9300000071525574
epoch:   14	Loss:0.243	Acc:0.9100000262260437
epoch:   14	Loss:0.160	Acc:0.9300000071525574
epoch:   14	Loss:0.288	Acc:0.9300000071525574
epoch:   14	Loss:0.344	Acc:0.9300000071525574
epoch:   14	Loss:0.224	Acc:0.9100000262260437
epoch:   14	Loss:0.308	Acc:0.9100000262260437
epoch:   14	Loss:0.406	Acc:0.8999999761581421
epoch:   14	Loss:0.252	Acc:0.9200000166893005
epoch:   14	Loss:0.377	Acc:0.9200000166893005
epoch:   14	Loss:0.246	Acc:0.949999988079071
epoch:   14	Loss:0.230	Acc:0.9200000166893005
epoch:   14	Loss:0.263	Acc:0.9300000071525574
epoch:   14	Loss:0.360	Acc:0.9399999976158142
epoch:   14	Loss:0.250	Acc:0.9200000166893005
epoch:   14	Loss:0.346	Acc:0.9200000166893005
epoch:   14	Loss:0.109	Acc:0.9700000286102295
epoch:   14	Loss:0.256	Acc:0.9200000166893005
epoch:   14	Loss:0.242	Acc:0.9300000071525574
epoch:   14	Loss:0.643	Acc:0.8600000143051147
epoch:   14	Loss:0.152	Acc:0.949999

In [30]:
print("정확도 : ", sess.run(accuracy, feed_dict={X:mnist.test.images, 
                                             y:mnist.test.labels}))

AttributeError: 'Datasets' object has no attribute 'images'

## XOR 해결
- 하나의 인공신경망으로는 해결이안되고 